In [ ]:
from pyspark import SparkContext
from scipy import sparse as sm
from sklearn.preprocessing import normalize
import numpy as np
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import spearmanr
from scipy.stats import pearsonr as pears
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
import time
from sparsesvd import sparsesvd
import math as mt
sc = SparkContext.getOrCreate()

In [ ]:
train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1])))
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(','))

test_users=test_clean_data.map( lambda x: int(x[0])).collect()


grouped_rates = train_clean_data.filter(lambda x: x[0] in test_users).map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
grouped_rates_dic = dict(grouped_rates)


item_ratings = train_clean_data.map(lambda x: (x[0], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))
user_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1]))).collect()
user_ratings_mean_dic=dict(user_ratings_mean)


item_ratings_forTop = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
shrinkage_factor = 5
item_ratings_mean = item_ratings_forTop.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()


#matrix interactions with global effect creation
users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
UxI_basic= sm.csr_matrix((ratings, (users, items)))

#remove global effect
global_mean = train_clean_data.map(lambda x: x[2]).mean()

trainr2 = train_clean_data.map(lambda x: (x[0], x[1], x[2]-global_mean))
temp = trainr2.map(lambda x: (x[1],(x[2],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
i_Bias = temp.collectAsMap()

trainr3 = trainr2.map(lambda x: (x[0],x[1], x[2] - i_Bias.get(x[1],0)))
temp2 = trainr3.map(lambda x: (x[0],(x[2],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
u_Bias = temp2.collectAsMap()

train_clean_data = train_clean_data.map(lambda x: (x[0], x[1], x[2] - i_Bias.get(x[1],0) - u_Bias.get(x[0],0)))
train_clean_data.take(5)

#matrix interactions without global effect creation
users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
UxI_global1= sm.csr_matrix((ratings, (users, items)))

#matrix icm creations
items_for_features= icm_clean_data.map(lambda x:x[0]).collect()
features = icm_clean_data.map(lambda x:x[1]).collect()
items_for_features.append(37142)
features.append(0)
ones=[1.0]*len(items_for_features)
IxF= sm.csr_matrix((ones, (items_for_features, features)))

In [ ]:
n_users,n_items=UxI_basic.shape
n_features=IxF.shape[1]

From now on we stopped using RDDs because of too high resource requiring of matrices calculations with them.

SVD

In [ ]:
'''matrix factorization -  TUNE PARAMETER HERE'''
K = 800
%time U, S, Vt = sparsesvd(UxI_global1.tocsc(), K)
S=np.sqrt(S.data)
S=np.diag(S)

In [ ]:
'''calc predictions'''
%time UxI_pred_SVD=sm.csr_matrix(U.T.dot(S).dot(Vt))

Content Based

In [ ]:
IDF=[0]*n_features
for i in tqdm(range(n_features)):
    IDF[i]=np.log10(n_items/len(IxF.getcol(i).nonzero()[1]))
%time IxF=normalize(IxF,axis=1) #apply TF
%time IxF_idf=IxF.multiply(IDF) #apply IDF
%time UxF=UxI_global1.dot(IxF_idf) #calc users profiles

In [ ]:
'''calc predictions'''
%time UxI_pred_CB=UxF.dot(IxF.T)
del IxF

Item Based

In [ ]:
'''calc item similarities based on features'''
%time IxI_sim_f=IxF_idf.dot(IxF_idf.T)
IxI_sim_f.setdiag(0)

In [ ]:
'''calc item similarities based on collaborative'''
%time UxI_glob_unbiased=UxI_global1.copy().tolil()
bias=sm.lil_matrix((n_users,n_items))
for usr in tqdm(range(n_users)):
    bias[usr,UxI_global1.getrow(usr).nonzero()]=(UxI_global1[usr,:].data.mean())
UxI_glob_unbiased-=bias
UxI_glob_unbiased=UxI_glob_unbiased.tocsr()
%time IxI_sim_c=UxI_glob_unbiased.T.dot(UxI_glob_unbiased) #"pearson" similarity without denominator
IxI_sim_c.setdiag(0)
del bias

In [ ]:
'''merge item similarities -  TUNE PARAMETER HERE'''
%time IxI_sim = IxI_sim_f + IxI_sim_c*0.0000005
del IxI_sim_f
del IxI_sim_c

In [ ]:
'''take knn items -  TUNE PARAMETER HERE'''
IxI_sim_knn=sm.lil_matrix((n_items,n_items))
k=200
for i in tqdm(range(n_items)):    
    top_k_idx =IxI_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    IxI_sim_knn[i,top_k_idx]=IxI_sim[i,top_k_idx]
del IxI_sim 

In [ ]:
'''calc predictions'''
%time UxI_pred_CI=UxI_global1.dot(IxI_sim_knn.T)
del IxI_sim_knn

User Based

In [ ]:
'''calc user similarities based on profiles'''
%time UxU_sim_p=UxF.dot(UxF.T) #numerators of cosine
%time cos=sm.csr_matrix(cosine_similarity(UxF)) #cosine
cos.data=1/cos.data
denominators=UxU_sim_p.multiply(cos) #get denominators of cosine
del cos
denominators.data+=1 #add shrinkage to cosine denominator
denominators.data=1/denominators.data
%time UxU_sim_p=UxU_sim_p.multiply(denominators) #calc shrinked cosine
del denominators
UxU_sim_p.setdiag(0)

In [ ]:
'''calc user similarities based on collaborative'''
%time UxU_sim_c=UxI_glob_unbiased.dot(UxI_glob_unbiased.T) #pearson similarity only numerator
UxU_sim_c.setdiag(0)
del UxI_glob_unbiased

In [ ]:
'''merge user similarities -  TUNE PARAMETER HERE'''
%time UxU_sim = UxU_sim_p + UxU_sim_c*0.0000005
del UxU_sim_p
del UxU_sim_c

In [ ]:
'''take knn users -  TUNE PARAMETER HERE'''
UxU_sim_knn=sm.lil_matrix((n_users,n_users))
k=100
for i in tqdm(range(n_users)):    
    top_k_idx =UxU_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    UxU_sim_knn[i,top_k_idx]=UxU_sim[i,top_k_idx]  
del UxU_sim

In [ ]:
'''calc_predictions'''
%time UxI_pred_CU=UxU_sim_knn.dot(UxI_global1)

In [ ]:
del UxU_sim_knn
del UxI_basic
del UxI_global1
del UxF
del IxF_idf

Removal of already voted items, not mandatory

In [ ]:
#create matrix for test users predictions
UxI_pred_CB_test=sm.lil_matrix((n_users,n_items))
UxI_pred_CI_test=sm.lil_matrix((n_users,n_items))
UxI_pred_CU_test=sm.lil_matrix((n_users,n_items))
UxI_pred_SVD_test=sm.lil_matrix((n_users,n_items))

In [ ]:
#take only test users predictions
%time UxI_pred_CB_test[test_users,:]=UxI_pred_CB[test_users,:]
%time UxI_pred_CI_test[test_users,:]=UxI_pred_CI[test_users,:]
%time UxI_pred_CU_test[test_users,:]=UxI_pred_CU[test_users,:]
%time UxI_pred_SVD_test[test_users,:]=UxI_pred_SVD[test_users,:]

In [ ]:
#remove items already voted
for user in tqdm(test_users):
    UxI_pred_CB_test[user,grouped_rates_dic[user]]=0
    UxI_pred_CI_test[user,grouped_rates_dic[user]]=0
    UxI_pred_CU_test[user,grouped_rates_dic[user]]=0
    UxI_pred_SVD_test[user,grouped_rates_dic[user]]=0

In [ ]:
#convert to csr to improve future operations performances
%time UxI_pred_CB_test=UxI_pred_CB_test.tocsr()
%time UxI_pred_CI_test=UxI_pred_CI_test.tocsr()
%time UxI_pred_CU_test=UxI_pred_CU_test.tocsr()
%time UxI_pred_SVD_test=UxI_pred_SVD_test.tocsr()

In [ ]:
del UxI_pred_CB
del UxI_pred_CI
del UxI_pred_CU
del UxI_pred_SVD

Algorithms adjusted with svd in order to remove possible even scores and conseguent random ordering

In [ ]:
%time UxI_pred_CB_SVD = UxI_pred_CB_test + UxI_pred_SVD_test*180.0 #actually not this one

In [ ]:
%time UxI_pred_CI_SVD = UxI_pred_CI_test + UxI_pred_SVD_test*0.000001

In [ ]:
%time UxI_pred_CU_SVD = UxI_pred_CU_test + UxI_pred_SVD_test*0.000001

In [ ]:
del UxI_pred_CB_test
del UxI_pred_CI_test
del UxI_pred_CU_test
del UxI_pred_SVD_test

In [ ]:
'''TUNE PARAMETERS HERE'''
def bordaLikeAggr(rank1,rank2,rank3):    
    result=[0]*n_items
    rg=150
    for i in range(rg):
        item1=rank1.argmax()
        item2=rank2.argmax()
        item3=rank3.argmax()   
        
        if rank1[item1]>0.0:
            result[item1]+=(6/(i+1))
        rank1[item1]=-500

        if rank2[item2]>0.0:
            result[item2]+=(6/(i+1))
        rank2[item2]=-500
        
        if rank3[item3]>0.0:
            result[item3]+=(0.1/(i+1))
        rank2[item3]=-500
    
    return sm.csr_matrix(result)

In [ ]:
f = open('submission_RangAgg_6-6-01.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in tqdm(test_users):
    top=[0,0,0,0,0]

    user_predictions=bordaLikeAggr(UxI_pred_CI_SVD.getrow(user).toarray()[0],UxI_pred_CB_SVD.getrow(user).toarray()[0],UxI_pred_CU_SVD.getrow(user).toarray()[0])
    iterator = 0
    for i in range(5):
        prediction = user_predictions.argmax()
        while prediction in grouped_rates_dic[user] and prediction != 0:
            user_predictions[0,prediction]=-9
            prediction=user_predictions.argmax()
        if prediction == 0:
            prediction = item_ratings_mean[iterator]
            while prediction in grouped_rates_dic[user] or prediction in top:
                iterator += 1
                prediction = item_ratings_mean[iterator]
            iterator += 1
        else:
            user_predictions[0,prediction]=-9
        top[i]=prediction    
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()